In [240]:
import pandas as pd
import re
from scipy import stats as st
import nltk 
from gensim import corpora
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import TfidfModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

In [149]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /home/max/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/max/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/max/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/max/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Задание 1

Ответьте на следующие вопросы:

* Как критики относятся к спортивным играм?
* Критикам нравятся больше игры на PC или на PS4?
* Критикам больше нравятся стрелялки или стратегии?

Для каждого вопроса:

сформулируйте нулевую и альтернативную гипотезы;
выберите пороговый уровень статистической значимости;
опишите полученные результаты статистического теста.

In [150]:
url_games = 'https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/stat_case_study/vgsales.csv'

In [151]:
df_games = pd.read_csv(url_games)
df_games

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


## 1.1 Как критики относятся к спортивным играм?
* H0: Удовлетворенность критиков спортивными играми < 80% 
* H1: Удовлетворенность критиков спортивными играми >= 80% 

In [152]:
sport_critic = df_games.loc[df_games['Genre'] == 'Sports', ['Name','Critic_Score']]
sport_critic

,Name,Critic_Score
0,Wii Sports,76.0
3,Wii Sports Resort,80.0
13,Wii Fit,80.0
15,Wii Fit Plus,80.0
77,FIFA 16,82.0
...,...,...
16680,G1 Jockey 4 2008,NaN
16692,Outdoors Unleashed: Africa 3D,NaN
16697,PGA European Tour,NaN
16704,Mezase!! Tsuri Master DS,NaN


In [153]:
sport_critic.dropna(inplace=True)
sport_critic

,Name,Critic_Score
0,Wii Sports,76.0
3,Wii Sports Resort,80.0
13,Wii Fit,80.0
15,Wii Fit Plus,80.0
77,FIFA 16,82.0
...,...,...
16456,NHL 2K7,78.0
16478,John Daly's ProStroke Golf,57.0
16532,Pro Evolution Soccer 2010,78.0
16550,Wade Hixton's Counter Punch,78.0


### У половины спортивных игр просто отсутствуют оценки, ни о какой репрезентативности выборки речи не идет и выводы не могут быть верными но я продолжу задание ради практики

In [154]:
alpha = 0.05
result = st.ttest_1samp(sport_critic['Critic_Score'], 80, alternative='greater')

print(sport_critic['Critic_Score'].mean())
print(result.pvalue)

if result.pvalue < alpha: 
    print('Отвергаем нулевую гипотезу, среднее больше 80 (продукция нравится)')
else:
    print('Не отвергаем нулевую нулевую гипотезу, критикам не нравится спортивные игры')

71.96817420435511
1.0
Не отвергаем нулевую нулевую гипотезу, критикам не нравится спортивные игры


## Но как я и написал выше выводы ничего не стоят

## 1.2 Критикам нравятся больше игры на PC или на PS4?
* H0: Оценки критиков на игры на PC = PS4
* H1: Оценки критиков на игры на PC != PS4

In [155]:
pc_critic = df_games.loc[df_games['Platform'] == 'PC', ['Name','Critic_Score']]
pc_critic

,Name,Critic_Score
85,The Sims 3,86.0
138,World of Warcraft,93.0
192,Diablo III,88.0
195,Microsoft Flight Simulator,NaN
218,StarCraft II: Wings of Liberty,93.0
...,...,...
16694,Pony Friends 2,NaN
16696,Metal Gear Solid V: Ground Zeroes,80.0
16700,Breach,61.0
16706,STORM: Frontline Nation,60.0


In [156]:
pc_critic.dropna(inplace=True)
pc_critic

,Name,Critic_Score
85,The Sims 3,86.0
138,World of Warcraft,93.0
192,Diablo III,88.0
218,StarCraft II: Wings of Liberty,93.0
284,Half-Life,96.0
...,...,...
16644,Hospital Tycoon,51.0
16696,Metal Gear Solid V: Ground Zeroes,80.0
16700,Breach,61.0
16706,STORM: Frontline Nation,60.0


In [157]:
ps_critic = df_games.loc[df_games['Platform'] == 'PS4', ['Name','Critic_Score']]
ps_critic

,Name,Critic_Score
31,Call of Duty: Black Ops 3,NaN
42,Grand Theft Auto V,97.0
77,FIFA 16,82.0
87,Star Wars Battlefront (2015),NaN
92,Call of Duty: Advanced Warfare,83.0
...,...,...
16504,Root Letter,69.0
16507,Shin Hayarigami 2,NaN
16530,Dungeons 2,61.0
16534,Carmageddon: Max Damage,51.0


In [158]:
ps_critic.dropna(inplace=True)
ps_critic

,Name,Critic_Score
42,Grand Theft Auto V,97.0
77,FIFA 16,82.0
92,Call of Duty: Advanced Warfare,83.0
94,FIFA 17,85.0
105,Fallout 4,87.0
...,...,...
16490,Super Dungeon Bros,42.0
16503,Sherlock Holmes: The Devil's Daughter,70.0
16504,Root Letter,69.0
16530,Dungeons 2,61.0


## Так же абсолютно абсурдное количество пропущенных оценок что бы делать вывод о ситуации в целом

In [159]:
alpha = 0.05
result = st.ttest_ind(pc_critic['Critic_Score'], ps_critic['Critic_Score'], equal_var=False)
print(result)
print(pc_critic['Critic_Score'].mean())
print(ps_critic['Critic_Score'].mean())

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, критики не считают игры равно хорошими')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=4.3087588262138725, pvalue=2.067249157283479e-05)
75.92867132867133
72.09126984126983
Отвергаем нулевую гипотезу, критики не считают игры равно хорошими


## Так как оценки РС выше, а статистически выборки различаются можно было бы сделать выводы о том, что игры на РС ценятся выше, если бы не такие пробелы в данных

## 1.3 Критикам больше нравятся стрелялки или стратегии?
* H0: Оценки критиков на жанр стрелялки = стратегии
* H1: Оценки критиков на жанр стрелялки != стратегии

In [160]:
shooter_critic = df_games.loc[df_games['Genre'] == 'Shooter', ['Name','Critic_Score']]
shooter_critic

,Name,Critic_Score
9,Duck Hunt,NaN
29,Call of Duty: Modern Warfare 3,88.0
31,Call of Duty: Black Ops 3,NaN
32,Call of Duty: Black Ops,87.0
34,Call of Duty: Black Ops II,83.0
...,...,...
16580,DoDonPachi Daifukkatsu: Black Label,NaN
16581,Space Raiders,40.0
16606,"Transformers: War for Cybertron (XBox 360, PS3...",NaN
16700,Breach,61.0


In [161]:
shooter_critic.dropna(inplace=True)
shooter_critic

,Name,Critic_Score
29,Call of Duty: Modern Warfare 3,88.0
32,Call of Duty: Black Ops,87.0
34,Call of Duty: Black Ops II,83.0
35,Call of Duty: Black Ops II,83.0
36,Call of Duty: Modern Warfare 2,94.0
...,...,...
16433,Greg Hastings' Tournament Paintball Max'd,70.0
16452,Inversion,59.0
16541,Clive Barker's Jericho,63.0
16581,Space Raiders,40.0


In [162]:
strategy_critic = df_games.loc[df_games['Genre'] == 'Strategy', ['Name','Critic_Score']]
strategy_critic

,Name,Critic_Score
168,Pokemon Stadium,NaN
206,Warzone 2100,NaN
218,StarCraft II: Wings of Liberty,93.0
273,Warcraft II: Tides of Darkness,NaN
339,Pokémon Trading Card Game,NaN
...,...,...
16621,Codename: Panzers Complete Collection,NaN
16644,Hospital Tycoon,51.0
16682,End of Nations,NaN
16685,Worms 2,NaN


In [163]:
strategy_critic.dropna(inplace=True)
strategy_critic

,Name,Critic_Score
218,StarCraft II: Wings of Liberty,93.0
582,Halo Wars,82.0
1078,Sid Meier's Civilization V,90.0
1095,StarCraft II: Heart of the Swarm,86.0
1128,Pikmin,89.0
...,...,...
16423,Battlestations: Midway,76.0
16457,Act of Aggression,71.0
16573,The Eye of Judgment: Legends,69.0
16644,Hospital Tycoon,51.0


In [164]:
alpha = 0.05
result = st.ttest_ind(shooter_critic['Critic_Score'], strategy_critic['Critic_Score'], equal_var=False)
print(result)
print(shooter_critic['Critic_Score'].mean())
print(strategy_critic['Critic_Score'].mean())

if (result.pvalue < alpha):
    print('Отвергаем нулевую гипотезу, критики не считают жанры одинаково хорошими')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=-2.2972408230640315, pvalue=0.021938989522304823)
70.18114406779661
72.08609271523179
Отвергаем нулевую гипотезу, критики не считают жанры одинаково хорошими


## Так как критики статистически не считают игры одинаковыми, на основе среднего результата можно было бы сделать вывод что стратегии в среднем имеют оценки выше чем шутеры

## Задание 2
<div calss="alert alert-block alert-warning">
    <b>
    Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные здесь) по признаку спама. Для этого:

    1) Привидите весь текст к нижнему регистру;
    2) Удалите мусорные символы;
    3) Удалите стоп-слова;
    4) Привидите все слова к нормальной форме;
    5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:

    from sklearn.feature_extraction.text import TfidfVectorizer
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df.Message)
    names = tfidf.get_feature_names()
    tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
    Можете поэкспериментировать с параметрами TfidfVectorizer;
    6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите random_state=42. Используйте train_test_split;
    7) Постройте модель логистической регрессии, укажите random_state=42, оцените ее точность на тестовых данных;
    8) Опишите результаты при помощи confusion_matrix;
    9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).
    </b>
</div>

In [202]:
url = 'https://raw.githubusercontent.com/obulygin/pyda_homeworks/master/stat_case_study/spam.csv'

In [203]:
spam_df = pd.read_csv(url)
spam_df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [204]:
spam_df['Message'] = spam_df['Message'].str.lower()
spam_df

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,this is the 2nd time we have tried 2 contact u...
5568,ham,will ü b going to esplanade fr home?
5569,ham,"pity, * was in mood for that. so...any other s..."
5570,ham,the guy did some bitching but i acted like i'd...


In [205]:
def untrash_symb(ds):
    """
    Удаление мусорных символов из колонки 'Message' принимаемоего DataSeries
    """
    return re.sub('[\W_]+', ' ', ds)

In [206]:
spam_df['Message'] = spam_df['Message'].apply(untrash_symb)
spam_df

,Category,Message
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives arou...
...,...,...
5567,spam,this is the 2nd time we have tried 2 contact u...
5568,ham,will ü b going to esplanade fr home
5569,ham,pity was in mood for that so any other suggest...
5570,ham,the guy did some bitching but i acted like i d...


In [207]:
df_text = pd.DataFrame.copy(spam_df)

In [208]:
stopwords_set = set(stopwords.words('english'))

In [209]:
def rem_stwords(ds):
    """
    Убираем стопслова из колконки принимаемого DataSeries
    """
    words_of_ds = re.findall(r'\w+', ds)
    return [word for word in words_of_ds if word not in stopwords_set]

In [210]:
spam_df['Message'] = spam_df['Message'].apply(rem_stwords)
spam_df

,Category,Message
0,ham,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"[nah, think, goes, usf, lives, around, though]"
...,...,...
5567,spam,"[2nd, time, tried, 2, contact, u, u, 750, poun..."
5568,ham,"[ü, b, going, esplanade, fr, home]"
5569,ham,"[pity, mood, suggestions]"
5570,ham,"[guy, bitching, acted, like, interested, buyin..."


In [211]:
wordnet_lemmatizer = WordNetLemmatizer()

In [212]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)


In [213]:
def lem_this_sh(ds):
    """
    Леммизация слов принимаемого DataSeries
    """
    return [wordnet_lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in ds]
    

In [214]:
lem_this_sh(['guy', 'bitching', 'acted', 'like', 'interested', 'buyin'])

['guy', 'bitching', 'act', 'like', 'interested', 'buyin']

In [215]:
spam_df['Message'] = spam_df['Message'].apply(lem_this_sh)

## Без категорирования как часть речи леммизация была очень слабая но и тут конечно не иделальная, но получше

In [216]:
dictionary = corpora.Dictionary(spam_df.Message)
dict(dictionary)

{0: 'amore',
 1: 'available',
 2: 'buffet',
 3: 'bugis',
 4: 'cine',
 5: 'crazy',
 6: 'e',
 7: 'get',
 8: 'go',
 9: 'great',
 10: 'jurong',
 11: 'la',
 12: 'n',
 13: 'point',
 14: 'wat',
 15: 'world',
 16: 'joking',
 17: 'lar',
 18: 'ok',
 19: 'oni',
 20: 'u',
 21: 'wif',
 22: '08452810075over18',
 23: '2',
 24: '2005',
 25: '21st',
 26: '87121',
 27: 'apply',
 28: 'c',
 29: 'comp',
 30: 'cup',
 31: 'entry',
 32: 'fa',
 33: 'final',
 34: 'free',
 35: 'may',
 36: 'question',
 37: 'rate',
 38: 'receive',
 39: 'std',
 40: 'text',
 41: 'tkts',
 42: 'txt',
 43: 'win',
 44: 'wkly',
 45: 'already',
 46: 'dun',
 47: 'early',
 48: 'hor',
 49: 'say',
 50: 'around',
 51: 'life',
 52: 'nah',
 53: 'think',
 54: 'though',
 55: 'usf',
 56: '1',
 57: '3',
 58: '50',
 59: 'back',
 60: 'chgs',
 61: 'darling',
 62: 'freemsg',
 63: 'fun',
 64: 'hey',
 65: 'like',
 66: 'rcv',
 67: 'send',
 68: 'still',
 69: 'tb',
 70: 'week',
 71: 'word',
 72: 'xxx',
 73: 'aid',
 74: 'brother',
 75: 'even',
 76: 'patent',


In [217]:
def vectorize_message(ds):
    """Векторизация сообщения согласно составленнному словарю"""
    dictionary_ds = dictionary.doc2bow(ds)
    tfidf_matrix = tfidf.fit_transform(dictionary)
    names = tfidf.get_feature_names()
    tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
    print(tfidf_matrix)

In [220]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_text.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
tfidf_matrix


/home/max/.local/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5567,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5568,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5569,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5570,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [222]:
X = tfidf_matrix
y = spam_df.Category

In [223]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [225]:
clf = LogisticRegression(random_state=42).fit(X_train, y_train)

In [228]:
predicted_text = clf.predict(X_test)
predicted_text

array(['ham', 'ham', 'ham', ..., 'ham', 'ham', 'ham'], dtype=object)

In [234]:
len(predicted_text)

1839

In [229]:
y_test

3245    ham
944     ham
1044    ham
2484    ham
812     ham
       ... 
4944    ham
3313    ham
3652    ham
14      ham
4758    ham
Name: Category, Length: 1839, dtype: object

In [230]:
clf.score(X_test, y_test)

0.9630233822729745

## Если все правильно сделано результаты оочень хорошие

In [243]:
resault_matrix = confusion_matrix(y_test, predicted_text, labels=["ham", "spam"])
resault_matrix

array([[1591,    2],
       [  66,  180]])

### Оказывается есть функция построения матрицы ошибок, круто!! Судя по всему результаты неплохие, можно посчитать критерии Accurancy, Precision и Recall! 
* 2 FP Очень неплохо, получается всего 2 спам сообщения расценены как нормальные.
* 66 FN сообщения ложно помечены как спам, цифра конечно большая, за то вероятность токсичных сообщений "крайне мала" 

In [245]:
spam_df = pd.read_csv(url)

In [248]:
spam_df["Predict"] = clf.predict(X)
spam_df.loc[spam_df['Category'] != spam_df['Predict'], :]

,Category,Message,Predict
5,spam,FreeMsg Hey there darling it's been 3 week's n...,ham
15,spam,"XXXMobileMovieClub: To use your credit, click ...",ham
19,spam,England v Macedonia - dont miss the goals/team...,ham
54,spam,SMS. ac Sptv: The New Jersey Devils and the De...,ham
68,spam,"Did you hear about the new ""Divorce Barbie""? I...",ham
...,...,...,...
5381,spam,You have 1 new message. Call 0207-083-6089,ham
5449,spam,"Latest News! Police station toilet stolen, cop...",ham
5466,spam,http//tms. widelive.com/index. wml?id=820554ad...,ham
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...,ham


## Почему то получилось больше, чем в матрице ошибок. Подскажите пожалуста почему так получилось